In [2]:
import pickle
import math
import numpy as np
import time
from collections import defaultdict

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor

In [3]:
AutoML_path = ""
log_folder = "alphad3m_log"

log_DDN = AutoML_path+log_folder+'/DDN_single_time'
log_pipeline_steps = AutoML_path+log_folder+'/pipeline_steps_class_02272022'

combination_name = 'DDN+OneHot'
log_target_test_pred_time = AutoML_path+log_folder+'/'+combination_name+'_target_test_pred_time_03102022'


In [4]:
DDN = pickle.load(open(log_DDN, "rb"))
print(len(DDN))

pipeline_steps_class = pickle.load(open(log_pipeline_steps, 'rb'))
print(len(pipeline_steps_class))



39
7811


In [5]:
target_test_pred_time = {}
data_names = list(DDN.keys())
data_embed = DDN

for data_index, data_name in enumerate(data_names):
    print("="*40)
    print(data_index, data_name)
    
    task_test = [data_name]
    candidate_score = DDN[data_name]
    prev_key = None
    dict_key = None
    len_task_train = 0
    prev_len_task_train = 0
    for threshold in np.arange(0.05, 0.5, 0.05):
        prev_key = dict_key
        dict_key = '_'.join([data_name, combination_name, str(round(threshold,2))])
        
        start = time.time()
        task_train = [i for i in data_names if i!=data_name and abs(DDN[i]-candidate_score)<threshold]
        prev_len_task_train = len_task_train
        len_task_train = len(task_train)
        if len_task_train == 0:
            target_test_pred_time[dict_key] = 'N.A.'
            pickle.dump(target_test_pred_time, open(log_target_test_pred_time, "wb"))
            continue
            
        if len_task_train == prev_len_task_train:
            print(prev_key, dict_key)
            print(prev_len_task_train, len_task_train)
            target_test_pred_time[dict_key] =  target_test_pred_time[prev_key]
            continue

        X_train = np.array([np.hstack((data_embed[i[0]], i[1])) for i in pipeline_steps_class if i[0] in task_train])
        y_train = np.array([i[3] for i in pipeline_steps_class if i[0] in task_train])
        X_test = np.array([np.hstack((data_embed[i[0]], i[1])) for i in pipeline_steps_class if i[0] in task_test])
        y_test = np.array([i[3] for i in pipeline_steps_class if i[0] in task_test])
        print("X_train shape:", X_train.shape, "y_train shape:", y_train.shape)
        print("X_test shape:", X_test.shape, "y_test shape:", y_test.shape)

        model = MLPRegressor(hidden_layer_sizes=(256,128,64), learning_rate='adaptive', random_state=19)
        model.fit(X_train, y_train)
        end = time.time()
        duration = end-start

        y_pred = model.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        print("Threshold", threshold, "MAE:", mae, "MSE:", mse)
        print("Time:", duration)
        target_test_pred_time[dict_key] = (y_test, y_pred, duration, mae, mse)
        pickle.dump(target_test_pred_time, open(log_target_test_pred_time, 'wb'))
    


0 adult
X_train shape: (1526, 37) y_train shape: (1526,)
X_test shape: (452, 37) y_test shape: (452,)
Threshold 0.05 MAE: 0.1123323642799455 MSE: 0.024389497660162088
Time: 0.49362921714782715
X_train shape: (3405, 37) y_train shape: (3405,)
X_test shape: (452, 37) y_test shape: (452,)
Threshold 0.1 MAE: 0.06914698993994284 MSE: 0.016638473128178764
Time: 1.9894590377807617
X_train shape: (5415, 37) y_train shape: (5415,)
X_test shape: (452, 37) y_test shape: (452,)
Threshold 0.15000000000000002 MAE: 0.11329970783920833 MSE: 0.02812003687537896
Time: 2.40151309967041
X_train shape: (5892, 37) y_train shape: (5892,)
X_test shape: (452, 37) y_test shape: (452,)
Threshold 0.2 MAE: 0.12775849858122465 MSE: 0.032847400600078305
Time: 2.9129951000213623
X_train shape: (6745, 37) y_train shape: (6745,)
X_test shape: (452, 37) y_test shape: (452,)
Threshold 0.25 MAE: 0.16479666403311172 MSE: 0.045430489424858604
Time: 4.175504922866821
X_train shape: (6931, 37) y_train shape: (6931,)
X_test sh

Threshold 0.1 MAE: 0.06642734211465011 MSE: 0.00817984722992692
Time: 2.0732412338256836
X_train shape: (5708, 37) y_train shape: (5708,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.15000000000000002 MAE: 0.08926733031492702 MSE: 0.012622382077752894
Time: 3.291149854660034
X_train shape: (6128, 37) y_train shape: (6128,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.2 MAE: 0.06819570039046231 MSE: 0.00876640307059963
Time: 10.82988977432251
X_train shape: (6981, 37) y_train shape: (6981,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.25 MAE: 0.07558209908404041 MSE: 0.009921762618205678
Time: 4.349292993545532
X_train shape: (7167, 37) y_train shape: (7167,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.3 MAE: 0.08115716140070087 MSE: 0.01075225912884819
Time: 15.253896236419678
Australian_DDN+OneHot_0.3 Australian_DDN+OneHot_0.35
35 35
X_train shape: (7383, 37) y_train shape: (7383,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 

Threshold 0.25 MAE: 0.09412155565336282 MSE: 0.027375075814472273
Time: 9.009917259216309
X_train shape: (6785, 37) y_train shape: (6785,)
X_test shape: (196, 37) y_test shape: (196,)
Threshold 0.3 MAE: 0.08919517678326525 MSE: 0.027362342277024918
Time: 10.275615930557251
X_train shape: (7006, 37) y_train shape: (7006,)
X_test shape: (196, 37) y_test shape: (196,)
Threshold 0.35000000000000003 MAE: 0.08763892227348696 MSE: 0.026185520639110123
Time: 8.556665897369385
X_train shape: (7187, 37) y_train shape: (7187,)
X_test shape: (196, 37) y_test shape: (196,)
Threshold 0.4 MAE: 0.09502145241874556 MSE: 0.028323145151372457
Time: 7.905894994735718
X_train shape: (7403, 37) y_train shape: (7403,)
X_test shape: (196, 37) y_test shape: (196,)
Threshold 0.45 MAE: 0.08597581658439783 MSE: 0.02496499664145131
Time: 2.932888984680176
11 connect-4
X_train shape: (1694, 37) y_train shape: (1694,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.05 MAE: 0.20054716810247106 MSE: 0.0489322

Threshold 0.3 MAE: 0.4382641694802927 MSE: 0.20349998261166677
Time: 3.6702542304992676
X_train shape: (7403, 37) y_train shape: (7403,)
X_test shape: (204, 37) y_test shape: (204,)
Threshold 0.35000000000000003 MAE: 0.34423793808398484 MSE: 0.12986565281274556
Time: 5.566831827163696
dionis_DDN+OneHot_0.35 dionis_DDN+OneHot_0.4
37 37
X_train shape: (7607, 37) y_train shape: (7607,)
X_test shape: (204, 37) y_test shape: (204,)
Threshold 0.45 MAE: 0.4331913859713162 MSE: 0.1973834470425489
Time: 4.063852071762085
16 fabert
X_train shape: (665, 37) y_train shape: (665,)
X_test shape: (188, 37) y_test shape: (188,)
Threshold 0.05 MAE: 0.13546537198888292 MSE: 0.0262237647034157
Time: 0.3056797981262207
X_train shape: (1919, 37) y_train shape: (1919,)
X_test shape: (188, 37) y_test shape: (188,)
Threshold 0.1 MAE: 0.12537842943344657 MSE: 0.021176386535802474
Time: 0.7480959892272949
X_train shape: (2977, 37) y_train shape: (2977,)
X_test shape: (188, 37) y_test shape: (188,)
Threshold 0.1

Threshold 0.15000000000000002 MAE: 0.11415176992418649 MSE: 0.014355625608524488
Time: 1.642387866973877
jannis_DDN+OneHot_0.15 jannis_DDN+OneHot_0.2
15 15
X_train shape: (4935, 37) y_train shape: (4935,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.25 MAE: 0.1521211316236691 MSE: 0.02684424543133441
Time: 6.853058099746704
X_train shape: (5756, 37) y_train shape: (5756,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.3 MAE: 0.06653596102486421 MSE: 0.006726100448668422
Time: 7.669067144393921
X_train shape: (6884, 37) y_train shape: (6884,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.35000000000000003 MAE: 0.08336891830721861 MSE: 0.008833742644905865
Time: 7.894777774810791
X_train shape: (7595, 37) y_train shape: (7595,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.4 MAE: 0.08859817680913419 MSE: 0.010211428574734194
Time: 8.428193092346191
jannis_DDN+OneHot_0.4 jannis_DDN+OneHot_0.45
38 38
22 jasmine
X_train shape: (1274, 37) y_train s

Threshold 0.35000000000000003 MAE: 0.12842742251738076 MSE: 0.04397798566833418
Time: 5.264671087265015
X_train shape: (6931, 37) y_train shape: (6931,)
X_test shape: (266, 37) y_test shape: (266,)
Threshold 0.4 MAE: 0.14988957253669685 MSE: 0.04072701574396188
Time: 10.58340573310852
X_train shape: (7117, 37) y_train shape: (7117,)
X_test shape: (266, 37) y_test shape: (266,)
Threshold 0.45 MAE: 0.12499472673440735 MSE: 0.040906086900307614
Time: 4.716324090957642
27 mfeat-factors
X_train shape: (1203, 37) y_train shape: (1203,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.05 MAE: 0.07364852203060532 MSE: 0.014563340603694852
Time: 0.7181389331817627
X_train shape: (2240, 37) y_train shape: (2240,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.1 MAE: 0.06921996294174507 MSE: 0.012148447589771303
Time: 0.8908650875091553
X_train shape: (3822, 37) y_train shape: (3822,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.15000000000000002 MAE: 0.0739591613717

Threshold 0.45 MAE: 0.05112977444627499 MSE: 0.005265452786174086
Time: 6.356205940246582
32 riccardo
X_train shape: (1771, 37) y_train shape: (1771,)
X_test shape: (60, 37) y_test shape: (60,)
Threshold 0.05 MAE: 0.09552329030998875 MSE: 0.024018496236244906
Time: 1.103269100189209
X_train shape: (2396, 37) y_train shape: (2396,)
X_test shape: (60, 37) y_test shape: (60,)
Threshold 0.1 MAE: 0.09665336444909962 MSE: 0.023809098474688328
Time: 1.1652050018310547
X_train shape: (4374, 37) y_train shape: (4374,)
X_test shape: (60, 37) y_test shape: (60,)
Threshold 0.15000000000000002 MAE: 0.098282103267515 MSE: 0.02464643106191432
Time: 2.4609522819519043
riccardo_DDN+OneHot_0.15 riccardo_DDN+OneHot_0.2
20 20
X_train shape: (5432, 37) y_train shape: (5432,)
X_test shape: (60, 37) y_test shape: (60,)
Threshold 0.25 MAE: 0.0931239972138079 MSE: 0.022786291909156906
Time: 3.072545051574707
X_train shape: (6284, 37) y_train shape: (6284,)
X_test shape: (60, 37) y_test shape: (60,)
Threshold 0

Threshold 0.1 MAE: 0.1354151680379846 MSE: 0.036426356740787105
Time: 5.916474103927612
X_train shape: (4530, 37) y_train shape: (4530,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.15000000000000002 MAE: 0.28066264129723495 MSE: 0.09745956650608495
Time: 6.202378988265991
X_train shape: (5120, 37) y_train shape: (5120,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.2 MAE: 0.17526553374907555 MSE: 0.05286029259450623
Time: 9.583697319030762
X_train shape: (6612, 37) y_train shape: (6612,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.25 MAE: 0.22697466669031902 MSE: 0.07291554822869081
Time: 9.01286792755127
X_train shape: (7383, 37) y_train shape: (7383,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.3 MAE: 0.1743961431579391 MSE: 0.05084612090608828
Time: 9.038412094116211
X_train shape: (7391, 37) y_train shape: (7391,)
X_test shape: (216, 37) y_test shape: (216,)
Threshold 0.35000000000000003 MAE: 0.21558628766566018 MSE: 0.0675824861430